In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [50]:
mm_to_in = 0.03937008
in_to_mm = 1.0 / mm_to_in
mph_to_mps = 0.44704
in_to_m = 0.0254
ft_lbs_to_Nm = 1.0 / 1.355818
oz_in_to_ft_lbs = 1.0 / 192.00000012288
oz_in_to_Nm = 1.0 / 141.61193227806

class Motor:
    # castle 171 pulled from Raz Shifrin's Speed Secrets
    kv = 1650
    power = 0.0
    kt = 0.0
    max_rpm = 56000.0
    resistance = 0.0856092660801802
    diam_mm = 47.6 # 1.87 in
    length_mm = 85.1 # 3.35 in
    surf_area_mmsq = 2.0 * math.pi*(diam_mm/2.0)**2 + math.pi*diam_mm*length_mm
    surface_area = surf_area_mmsq / 1000.0**2
        
    def __init__(self, kv=1580, ns=8):
        self.kv = kv
        self.ns = ns
        self.kt = 1352.0 / kv
        self.heat_generated = 0.0
        self.external_temp = 0.0
        self.current = 0.0
        self.volts = 0.0
        self.power = 0.0

    def calc_heat_generated(self, current):
        '''Estimate heat generated over this time step'''
        self.heat_generated = self.resistance * current**2
        return self.heat_generated
        
    def update_heat_model(self, air_speed, air_temp):
        '''Estimate current temperature based upon incoming heat and expected dissipation'''
        # calc the heat generated over the last tic
        self.calc_heat_generated()
        # calc the heat expelled due to convection on the motor
        self.calc_heat_expelled(air_speed, air_temp)
        
    def calc_heat_expelled(self, air_speed, air_temp):
        # convection equation
        #q = h_c * A * dT =  # heat transfered per unit time - watts
        A = self.surface_area # area of heat transfer surface - m^2
        print("surface area: ", self.surface_area, " m^2")
        h_c = 10.45 - air_speed + 10.0 * air_speed**0.5 # convective heat transfer coefficient of the process - W / (m^2 C)
        print("h_c: ", h_c)
        self.external_temp = 95.0
        dT = self.external_temp - air_temp # temperature difference between the surface and the fluid - C
        q = h_c * A * dT
        return q
        # convection needs to be >= internal power generated for whatever temperature we want
        
    def calc_motor_torque(self, current):
        '''Estimate total torque based on current'''
        t_oz_in = self.kt * current
        # print('t_oz_in: ', t_oz_in)
        # t_ft_lbs = t_oz_in * oz_in_to_ft_lbs
        # print("t_ft_lbs: ", t_ft_lbs)
        return t_oz_in * oz_in_to_Nm
    
    
    def calc_steady_state_heat_model(self):
        current = np.linspace(200, 375, 500)
        heat_generated = np.vectorize(self.calc_heat_generated)(current)
        plt.plot(current, heat_generated)
        plt.grid()
        plt.xlabel('Current (A)')
        plt.ylabel('Heat Generated (W)')
        plt.show()
        heat_expelled = np.vectorize(self.calc_heat_expelled)(air_speed = 62.0, air_temp = 30.0) 
        print("heat expelled: ", heat_expelled)
        # print("heat_flux: ", heat_flux)
        
    
    def estimate_internal_resistance(self):
        # this is for the 1717 and has data pulled from raz shifrin's youtube speed secrets finale
        # with 34/34 gearing
        run1_volts = np.array([26.4, 26.3, 30.2, 33.1]) # mouse, min, avg, max
        run1_amps = np.array([273.0, 0.0, 130.7, 276.4])
        run1_watss = np.array([7210, 0, 3820, 7273])
        run1_rpm = np.array([36682, 1251, 23890, 38963])
        # with 40/34 gearing
        run2_volts = np.array([26.9, 26.1, 30.9, 34.2]) # mouse, min, avg, max
        run2_amps = np.array([361.0, 25.8, 205.4, 361.0])
        run2_watss = np.array([9717.0, 817.0, 5979.0, 10203.0])
        run2_rpm = np.array([34497.0, 7926.0, 25495.0, 35569.0])
        
        # calc_resistance
        run1_resistance = run1_volts / run1_amps
        run2_resistance = run2_volts / run2_amps
        print("run1_resistance: ", run1_resistance)
        print("run2_resistance: ", run2_resistance)
        
        # mouse is the only one that is guaranteed 1:1
        print("average at mouse: ", np.mean(np.array([run1_resistance[0], run2_resistance[0]])))
        
        # calc watts
        e_r1_watts = run1_volts * run1_amps
        e_r2_watts = run2_volts * run2_amps
        
        # check watts are right
        print("run 1 e watts: ", e_r1_watts)
        print("run 1 a watts: ", run1_watss)
        print("run 2 e watts: ", e_r2_watts)
        print("run 2 a watts: ", run2_watss)
        
        
    
motor = Motor()
#motor.calc_steady_state_heat_model() - it appears it is effectively impossible to keep it cool - error hopefully?

t_motor = motor.calc_motor_torque(350.0)
print("torque: " , t_motor, " Nm")
t_wheel = t_motor * gearRatio.total_reduction_ratio
print(t_wheel)

torque:  2.1148900807172777  Nm
5.946647928960841


In [43]:
1.14 * 1.355818

1.5456325199999998

In [ ]:
class ESC:
    max_current = 0.0
    
    def __init__(self):
        pass      

In [8]:

class GearRatios:
    motor_rpm = 0.0
    pinion = 40.0#34.0
    spur = 34.0
    differential_ratio = 3.308
    tire_diam = 100.0 # mm
    tire_circumference = math.pi * tire_diam 
    
    
    def __init__(self):
        pass
    
    def calc_car_speed_mph(self, motor_rpm):
        # calculate car speed based on drive train config and motor RPM
        self.motor_rpm = motor_rpm
        self.spur_pinion_ratio = self.spur / self.pinion
        # print("spur_pinion ratio: ", self.spur_pinion_ratio)
        self.total_reduction_ratio = self.spur_pinion_ratio * self.differential_ratio
        # print("Total Reduction Ratio: ", self.total_reduction_ratio)
        # print("Tire Circumference: ", self.tire_circumference, " mm", self.tire_circumference * mm_to_in, " in")
        self.roll_out = self.tire_circumference / self.total_reduction_ratio
        # print("Rollout Ratio: ", self.roll_out * mm_to_in, " motor rotations / inch of travel")
        speed_mph = (motor_rpm * self.roll_out * mm_to_in * 60) / (12 * 5280)
        # speed_mps = speed_mph * mph_to_mps
        return speed_mph
        
    
    def calc_car_speed_mps(self, motor_rpm):
        # calculate car speed based on drive train config and motor RPM
        self.motor_rpm = motor_rpm
        self.spur_pinion_ratio = self.spur / self.pinion
        # print("spur_pinion ratio: ", self.spur_pinion_ratio)
        self.total_reduction_ratio = self.spur_pinion_ratio * self.differential_ratio
        # print("Total Reduction Ratio: ", self.total_reduction_ratio)
        # print("Tire Circumference: ", self.tire_circumference, " mm", self.tire_circumference * mm_to_in, " in")
        self.roll_out = self.tire_circumference / self.total_reduction_ratio
        # print("Rollout Ratio: ", self.roll_out * mm_to_in, " motor rotations / inch of travel")
        speed_mph = (motor_rpm * self.roll_out * mm_to_in * 60) / (12 * 5280)
        speed_mps = speed_mph * mph_to_mps
        return speed_mps
        
gearRatio = GearRatios()
print("speed in mph: ", gearRatio.calc_car_speed_mph(38963))#46768))
print("speed in mps: ", gearRatio.calc_car_speed_mps(38963))#46768))

speed in mph:  162.30063300498173
speed in mps:  72.55487497854703
